In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pprint import pprint

In [ ]:
pprint(plt.rcParams.keys(), compact=True)

In [ ]:
#plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14

plt.rcParams['figure.figsize'] = 14,4
plt.rcParams['figure.facecolor'] = (1,1,1,0)

plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.linewidth'] = 1

plt.rcParams['xtick.labelsize'] = "small"
plt.rcParams['ytick.labelsize'] = "small"

plt.rcParams['grid.alpha'] = 0.3

In [ ]:
pprint(plt.style.available, compact=True, width=80)

In [ ]:
mpl.colors.BASE_COLORS

In [ ]:
mpl_colors = list(mpl.colors.CSS4_COLORS.keys())

pprint(mpl_colors, compact=True, width=80)

In [ ]:
LINK = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv"

In [ ]:
quakes = pd.read_csv(LINK, parse_dates=['time', 'updated'])
quakes.info()

In [ ]:
quakes.head()

In [ ]:
quakes.shape

In [ ]:
quakes['timeframe'] = quakes['time'].apply(lambda x: str(x)[0:10])
quakes['starttime'] = quakes['time'].apply(lambda x: str(x)[0:19])
quakes['updatetime'] = quakes['updated'].apply(lambda x: str(x)[0:19])

quakes.filter(regex="time")

In [ ]:
quakes = quakes.loc[quakes['mag']>0]

In [ ]:
quakes.isna().sum()

In [ ]:
# completed cases only
quakes = quakes.dropna(how="any")
quakes.isna().sum()

In [ ]:
quakes.shape

In [ ]:
quakes = quakes.sort_values("time").reset_index(drop=True)
quakes.nunique()

In [ ]:
quakes.describe()

In [ ]:
quakes.head()

In [ ]:
quakes['mag'].max()

In [ ]:
fig, ax = plt.subplots(figsize=(14,8))

ax.spines['top'].set_visible(True)
ax.spines['right'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)

ax.set_xscale('log')
ax.set_yscale('log')

for row in quakes.query("locationSource=='pr'").itertuples():
    if row.mag > 4:
        plt.text(x=row.mag, y=row.depth+2, s=f"{row.place}={row.mag:,.1f}", va="bottom", ha="center")
        
        plt.scatter(x=row.mag, y=row.depth, alpha=0.8, c="firebrick")
    else:
        plt.scatter(x=row.mag, y=row.depth, alpha=0.4, c="goldenrod")
        
plt.xlabel("Magnitude", fontsize=12)
plt.ylabel("Depth", fontsize=12)
plt.grid(True, alpha=0.5)

plt.show();

In [ ]:
q = quakes['type'].value_counts(dropna=False)
q

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(aspect="equal"))

wedges, text, autotexts = ax.pie(
    q.values,
    autopct="%0.1f%%",
    shadow=True,
    textprops=dict(color="w")
)

ax.legend(wedges, q.index, title="locations", loc="center left", shadow=True, bbox_to_anchor=(1,0,0.5,1))

plt.setp(autotexts, size=12, weight="bold")

ax.set_title("Total Events")

plt.show();

In [ ]:
q = quakes.groupby('timeframe', as_index=False).count()
q.head()

In [ ]:
fig, ax = plt.subplots()
plt.xticks(rotation=90)
plt.grid(False)

ax.plot(q['timeframe'], q['id'], ls="dashed", lw=3, c="b", marker="s", markerfacecolor="k");

In [ ]:
q = quakes.groupby('timeframe', as_index=False).mean()
q.head()

In [ ]:
fig, ax = plt.subplots()
plt.xticks(rotation=90)

plt.grid(False)

plt.plot(q['timeframe'], q[['depth','mag','gap']], lw=1, marker="s")

plt.show();

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(10,10), sharey=False)

ax1.plot(q['timeframe'], q['depth'], "rs-")
ax1.set_ylabel("Depth")
ax1.set_xticks([])

ax2.plot(q['timeframe'], q['mag'], "bo--")
ax2.set_ylabel("Magnitude")
ax2.set_xticks([])

ax3.plot(q['timeframe'], q['gap'], "g^-.")
ax3.set_ylabel("Gap")

plt.xticks(rotation=90)

plt.show();

In [ ]:
q = quakes.groupby('timeframe').mean()
q.head()

In [ ]:
cols = ["dmin", "rms", "magError"]

plt.figure(figsize=(14,4))

ls = ['-','--','-.',':'] * 3
mk = ['o','v','^','<','>','8','s','*','H','h','D','d','P','p','X']
cs = ['b','g','r','c','m','y','k','w'] * 2

for i, X in enumerate(cols):
    q[X].plot(color=cs[i], ls=ls[i], marker=mk[i], markerfacecolor=cs[i+1], lw=3, markersize=8)
    
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.xticks(rotation=0)

plt.show();

In [ ]:
quakes['locationSource'].value_counts()

In [ ]:
q = quakes.query("locationSource=='pr'")
q.head()

In [ ]:
plt.figure(figsize=(10,10))

colors = np.where(q['status']=="automatic", "navy", "crimson")
plt.scatter(data=q, x="mag", y="gap", s="depth", marker="o", c=colors)
plt.xlabel("Magnitude")
plt.ylabel("Gap")

plt.show();

In [ ]:
q = quakes.pivot_table(index='timeframe', columns='type', values='mag', fill_value=0, aggfunc=np.mean).reset_index()

q

In [ ]:
dates = q.timeframe
values = q.earthquake

In [ ]:
plt.figure(figsize=(14,10))
plt.plot(dates, values, lw=4, alpha=0.3)
plt.scatter(dates, values, s=50, marker="s", c="blue")
plt.ylim(1, values.max() * 1.1)

for index, value in enumerate(values):
    plt.text(x=index, ha="center", y=value + .03, s=f"{value:,.3f}", rotation=45)
    
plt.hlines(y=np.mean(value), xmin=min(dates), xmax=max(dates), ls="--", color="grey")

plt.title("Earthquake Analysis", fontsize=20)
plt.xlabel("Dates")
plt.ylabel("Values")
plt.grid(False)
plt.xticks(rotation=90)

plt.show();

In [ ]:
q = quakes.groupby(['locationSource'], as_index=False).mean()

q

In [ ]:
sources = q['locationSource'].values
values = q['magNst'].values

plt.figure(figsize=(14,4))
plt.bar(sources, values, color="navy", alpha=0.5)
plt.ylim(0, values.max() * 1.1)

for sources, value in enumerate(values):
    plt.text(x=sources, ha="center", va="bottom", y=value, s=f"{value:,.1f}", fontsize=14)
    
plt.xlabel("Index", fontsize=16)
plt.ylabel("Values", fontsize=16)
plt.grid(False)
plt.xticks(rotation=0)

plt.show();

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(12,4))

q.plot(kind="barh", x='locationSource', y='depth', color="b", ax=ax0)
ax0.set(title="Average Depth", xlabel="", ylabel="Location Sources")
ax0.legend().set_visible(False)

q.plot(kind="barh", x='locationSource', y='mag', color="m", ax=ax1)
ax1.set(title="Average Magnitude", xlabel="", ylabel="")
ax1.legend().set_visible(False)

fig.suptitle("Earthquake Analysis", fontsize=14, fontweight="bold")

plt.tight_layout()

plt.show();

In [ ]:
q = quakes\
.groupby('locationSource')\
.size()\
.reset_index()\
.rename(columns={0:'size'})\
.sort_values('size', ascending=False)

q['cum_pct'] = round(q['size'].cumsum() / q['size'].sum() * 100, 2)

q

In [ ]:
from matplotlib.ticker import PercentFormatter

In [ ]:
fig, ax = plt.subplots(figsize=(14,4))

ax.bar(q['locationSource'], q['size'])
ax.set_title("Pareto Chart")
ax.set_xlabel("Location Sources")
ax.set_ylabel("Frequency")

ax1 = ax.twinx()
ax1.plot(q['locationSource'], q['cum_pct'], color="red", marker="o", ms=7, alpha=0.5)
ax1.axhline(80, color="orange", ls="--")
ax1.yaxis.set_major_formatter(PercentFormatter())
ax1.set_ylabel("Cumulative Percentage")

plt.show();